In [ ]:
import os
import pandas as pd
import csv
import requests
import time

def encontrar_arquivo_excel(pasta):
    arquivos = [f for f in os.listdir(pasta) if f.endswith('.xlsx')]
    if not arquivos:
        print("Nenhum arquivo Excel encontrado na pasta especificada.")
        return None
    
    arquivos.sort(key=lambda x: os.path.getmtime(os.path.join(pasta, x)), reverse=True)
    return os.path.join(pasta, arquivos[0])

def extrair_cnpjs(arquivo_excel):
    try:
        df = pd.read_excel(arquivo_excel, dtype=str) 
    except Exception as e:
        print(f"Erro ao abrir o arquivo Excel: {e}")
        return []
    
    if "Nr. CNPJ/CPF" not in df.columns:
        print("A coluna 'Nr. CNPJ/CPF' não foi encontrada no arquivo Excel.")
        return []

   
    df["Nr. CNPJ/CPF"] = df["Nr. CNPJ/CPF"].astype(str).str.replace(r'\D', '', regex=True)
    df["Nr. CNPJ/CPF"] = df["Nr. CNPJ/CPF"].apply(lambda x: x.zfill(14) if len(x) <= 14 else x[-14:])

   
    return df["Nr. CNPJ/CPF"].drop_duplicates().tolist()[:1000]

def buscar_dados_cnpj(cnpj):
    """Consulta a API da ReceitaWS para obter dados do CNPJ."""
    url = f"https://www.receitaws.com.br/v1/cnpj/{cnpj}"
    try:
        response = requests.get(url)
    except Exception as e:
        print(f"Erro na requisição para o CNPJ {cnpj}: {e}")
        return None

    if response.status_code == 200:
        dados = response.json()
        return {
            "cnpj": dados.get("cnpj"),
            "nome": dados.get("nome"),
            "uf": dados.get("uf"),
            "municipio": dados.get("municipio"),
            "logradouro": dados.get("logradouro"),
            "numero": dados.get("numero"),
            "bairro": dados.get("bairro"),
            "cep": dados.get("cep"),
            "regime_tributario": dados.get("natureza_juridica"),
            "optante_simples": dados.get("opcao_pelo_simples") == "Sim",
            "optante_mei": dados.get("opcao_pelo_mei") == "Sim"
        }
    elif response.status_code == 429:
        print("Atingido o limite de taxa. Aguardando para continuar.")
        time.sleep(5)
        return buscar_dados_cnpj(cnpj)
    else:
        print(f"Erro ao consultar o CNPJ {cnpj}: Status Code {response.status_code}")
        return None

def salvar_dados_csv(dados, nome_arquivo):
    campos = [
        "cnpj", "nome", "uf", "municipio", "logradouro",
        "numero", "bairro", "cep", "regime_tributario",
        "optante_simples", "optante_mei"
    ]
    with open(nome_arquivo, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        for dado in dados:
            writer.writerow(dado)

def main():
    pasta_downloads = os.path.expanduser("~/Downloads")
    pasta_Nome_da_sua_pasta = os.path.join(pasta_downloads, "Nome_da_sua_pasta")
    
    arquivo_excel = encontrar_arquivo_excel(pasta_Nome_da_sua_pasta)
    if not arquivo_excel:
        return

    print(f"Arquivo encontrado: {arquivo_excel}")

    cnpjs = extrair_cnpjs(arquivo_excel)
    if not cnpjs:
        print("Nenhum CNPJ válido foi encontrado no arquivo.")
        return

    dados_coletados = []
    for i, cnpj in enumerate(cnpjs):
        print(f"Consultando CNPJ {i+1}/{len(cnpjs)}: {cnpj}")
        dados_cnpj = buscar_dados_cnpj(cnpj)
        if dados_cnpj:
            dados_coletados.append(dados_cnpj)
        time.sleep(1)

    if dados_coletados:
        nome_arquivo_csv = os.path.join(pasta_Nome_da_sua_pasta, 'D_F_EMPRESAS.csv')
        salvar_dados_csv(dados_coletados, nome_arquivo_csv)
        print(f"Dados de {len(dados_coletados)} CNPJs salvos em {nome_arquivo_csv}")
    else:
        print("Nenhum dado foi coletado.")

if __name__ == "__main__":
    main()
